In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import json

# Load intents from intent.json
with open('intent.json', 'r') as f:
    intents = json.load(f)

# Initialize the tokenizer and model (use a pre-trained BERT model fine-tuned for classification)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Replace with your fine-tuned model path if needed
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')  # Or path to your fine-tuned model

def get_intent(user_input):
    # Tokenize input
    inputs = tokenizer(user_input, return_tensors="pt")
    
    # Perform inference with the model
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the logits and predict the class
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    # Map predicted class to intent
    return intents['intents'][predicted_class]['intent']

/Users/yash/Library/Python/3.12/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import json
from rank_bm25 import BM25Okapi

# Load the menu data (assuming it's stored in a 'mockMenu 1.json' file)
with open('mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)["menu"]

# Preprocess the menu items (convert descriptions to list of tokens for BM25)
# Filter out menu items that have a 'None' description
menu_texts = [item['description'] for item in menu_data if item['description'] is not None]
menu_tokens = [text.split() for text in menu_texts]  # Tokenization of the valid menu descriptions

# Initialize BM25 model
bm25 = BM25Okapi(menu_tokens)

# Function to retrieve the most relevant menu item
def retrieve_menu_items(query):
    query_tokens = query.split()  # Tokenizing the user's query
    scores = bm25.get_scores(query_tokens)  # BM25 scores for each menu item based on the query
    best_idx = scores.argmax()  # Index of the highest-scoring menu item
    return menu_data[best_idx]  # Return the most relevant menu item

# Example usage
query = "healthy breakfast option"
result = retrieve_menu_items(query)
print(f"Most relevant item: {result['itemName']}\nDescription: {result['description']}")


Most relevant item: Rava Kichadi
Description: A healthy breakfast option made with semolina and vegetables lightly tempered with spices cooked to perfection


In [13]:
from transformers import pipeline

# Initialize DistilBERT for response generation
generator = pipeline("text-generation", model="distilbert-base-uncased")

def generate_response(intent, retrieved_item):
    if intent == "quickest_dish":
        prompt = f"The quickest dish is {retrieved_item['itemName']} which takes {retrieved_item['prepTimeInMins']} minutes to prepare."
    elif intent == "chef_special":
        prompt = f"Today's special is {retrieved_item['itemName']}. It is {retrieved_item['description']}."
    elif intent == "diet_recommendation":
        prompt = f"Based on your condition, I recommend {retrieved_item['itemName']}. {retrieved_item['description']}."
    elif intent == "price_inquiry":
        prompt = f"The price of {retrieved_item['itemName']} is ${retrieved_item['price']}."
    else:
        prompt = f"{retrieved_item['itemName']} is a great choice!"

    return generator(prompt, max_length=50)[0]['generated_text']


/Users/yash/Library/Python/3.12/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'DistilBertForMaskedLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextFor

In [16]:
from transformers import pipeline

# Initialize distilGPT-2 for response generation
generator = pipeline("text-generation", model="distilgpt2")

def generate_response(intent, retrieved_item):
    if intent == "quickest_dish":
        prompt = f"The quickest dish is {retrieved_item['itemName']} which takes {retrieved_item['prepTimeInMins']} minutes to prepare."
    elif intent == "chef_special":
        prompt = f"Today's special is {retrieved_item['itemName']}. It is {retrieved_item['description']}."
    elif intent == "diet_recommendation":
        prompt = f"Based on your condition, I recommend {retrieved_item['itemName']}. {retrieved_item['description']}."
    elif intent == "price_inquiry":
        prompt = f"The price of {retrieved_item['itemName']} is ${retrieved_item['price']}."
    else:
        prompt = f"{retrieved_item['itemName']} is a great choice!"

    # Generate response using distilGPT2
    generated = generator(prompt, max_length=50, num_return_sequences=1)[0]['generated_text']
    
    return generated

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [17]:
def chat(user_input):
    # Step 1: Intent Recognition using BERT
    intent = get_intent(user_input)
    
    # Step 2: Document Retrieval using BM25
    retrieved_item = retrieve_menu_items(user_input)
    
    # Step 3: Response Generation using DistilBERT
    response = generate_response(intent, retrieved_item)
    
    return response


In [18]:


user_input = "What is the quickest dish on your menu?"
response = chat(user_input)
print(response)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Baingan Ka Bharta is a great choice!
This book takes you on an astonishing journey through China, through India, through Australia, through Asia and beyond! I’ll tell you more.
This book is as close to as
